In [ ]:
import torch
import csv
import gzip
import nltk

In [ ]:
from nltk import sent_tokenize, word_tokenize, pos_tag

In [ ]:
text = "Machine learning is the science of getting computers to act without being explicitly programmed. In the past decade, machine learning has given us self-driving cars, practical speech recognition, effective web search, and a vastly improved understanding of the human genome. Machine learning is so pervasive today that you probably use it dozens of times a day without knowing it. Many researchers also think it is the best way to make progress towards human-level AI. In this class, you will learn about the most effective machine learning techniques, and gain practice implementing them and getting them to work for yourself. More importantly, you'll learn about not only the theoretical underpinnings of learning, but also gain the practical know-how needed to quickly and powerfully apply these techniques to new problems. Finally, you'll learn about some of Silicon Valley's best practices in innovation as it pertains to machine learning and AI."

In [ ]:
word_tokenize(text)

In [ ]:
word_tokenize("I'm cool.")

In [14]:
# Dictionary structure 
import gzip
import torch.nn as nn
from torch.autograd import Variable
from dpp_nets.utils.io import embd_iterator, make_embd
from collections import namedtuple
from nltk import word_tokenize

id_to_content = {}
errors = []
#embd_path = '/Users/Max/data/askubuntu/vectors_stackexchange.txt.gz'
embd_path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
embd_layer, word_to_ix  = make_embd(embd_path)
question = namedtuple('question', 'id title body title_ix body_ix')
path = '/Users/Max/data/askubuntu/text_tokenized.txt.gz'

In [13]:
import re
max_title_size = 0
max_body_size = 0
with gzip.open(path, 'rt') as f:
    
    for line in f:
        id, title, body = line.split("\t")
        id = int(id)
        if len(title) == 0:
            print(id)
            empty_cnt += 1
            continue
            
        #title = word_tokenize(title)
        #body = word_tokenize(body)
        title = title.split()
        body = body.split()

        # Create title_ix
        # title_ix = [word_to_ix[word] + 1 for word in title if word in word_to_ix.keys()]
        title_ix = []
        for word in title:
            if word in word_to_ix:
                ix = word_to_ix[word] + 1
                title_ix.append(ix)
            else:
                candidates = re.split('[;|,-/."]', word)
                for word in candidates:
                    if word in word_to_ix:
                        ix = word_to_ix[word] + 1
                        title_ix.append(ix)
        max_title_size = max(max_title_size, len(title_ix))
        
        # Create body_ix 
        # body_ix = [word_to_ix[word] + 1 for word in body if word in word_to_ix.keys()]
        body_ix = []
        for word in body:
            if word in word_to_ix:
                ix = word_to_ix[word] + 1
                body_ix.append(ix)
            else:
                candidates = re.split('[;|,-/."]', word)
                for word in candidates:
                    if word in word_to_ix:
                        ix = word_to_ix[word] + 1
                        body_ix.append(ix)
                    else:
                        errors.append(word)
                        
        max_body_size = max(max_body_size, len(body_ix))
        
        content = question(id, title, body, title_ix, body_ix)
        id_to_content[id] = content


print(errors)
print(set(errors))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [16]:
import re
max_title_size = 0
max_body_size = 0

with gzip.open(path, 'rt') as f:
    for line in f:
        id, title, body = line.split("\t")
        id = int(id)
        if len(title) == 0:
            print(id)
            empty_cnt += 1
            continue
            
        # title = word_tokenize(title)
        # body = word_tokenize(body)
        title = title.split()
        body = body.split()

        # Create title_ix
        title_ix = [word_to_ix[word] + 1 for word in title if word in word_to_ix.keys()]
        
        # Create body_ix 
        body_ix = [word_to_ix[word] + 1 for word in body if word in word_to_ix.keys()]
        error = [word for word in body if word not in word_to_ix.keys()]
        errors.extend(error)
        content = question(id, title, body, title_ix, body_ix)
        id_to_content[id] = content

print(errors)
print(set(errors))

KeyboardInterrupt: 

In [7]:
len(id_to_content)

92620

In [ ]:


path = '/Users/Max/data/askubuntu/train_random.txt'
i = 0
with open(path) as f:
    for line in f:
        #parts = line.split("\t") 
        #print('ID', parts[0])
        #print('Title:', parts[1])
        #print('Body:', parts[2])
        i += 1
        #if i > 25: 
            #break

In [ ]:
from collections import namedtuple

question = namedtuple('question', 'id title body title_ix body_ix')

In [ ]:
hi = question(12, 'abc', 'def', [3,4,5], [8,0])

In [ ]:
hi.body_ix

In [ ]:
import gzip
import numpy as np
path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
path = '/Users/Max/data/askubuntu/vectors_stackexchange.txt.gz'
e = embd_iterator(path)

In [ ]:
def embd_iterator(embd_path):
    with gzip.open(embd_path, 'rt') as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split()
                word = parts[0]
                embd = np.array([float(x) for x in parts[1:]])
                yield word, embd

In [ ]:
i = 0
for (word, embd) in e:
    print(word)
    print(embd)
    i += 1
    if i > 10:
        break

In [ ]:
from dpp_nets.utils.io import embd_iterator, make_embd
import torch
import torch.nn as nn
import numpy as np
from collections import OrderedDict
import gzip

In [ ]:
embd_path = '/Users/Max/data/askubuntu/vectors_stackexchange.txt.gz'
    
# Create dictionaries
ix_to_word = {}
ix_to_vecs = {}
word_to_ix = {}

for ix, (word, vecs) in enumerate(embd_iterator(embd_path)):
    ix_to_word[ix] = word
    ix_to_vecs[ix] = vecs
    word_to_ix[word] = ix

vocab_size, embd_dim = len(ix_to_word), len(ix_to_vecs[0])

In [ ]:
import torch
embd = torch.zeros(1 + vocab_size, embd_dim)

for i, vec in enumerate(ix_to_vecs.values(), 1):
    try:
        embd[i] = torch.FloatTensor(vec)
    except:
        a, b = i, vec
        print('failed')
        continue

embd_weight_dict = OrderedDict([('weight', embd)])

embd_layer = nn.Embedding(1 + vocab_size, embd_dim, padding_idx=0)
embd_layer.load_state_dict(embd_weight_dict)
embd_layer.weight.requires_grad = False

In [ ]:
a

In [ ]:
ix_to_word[a-1]

In [ ]:
embd_layer, word_to_ix  = make_embd(embd_path)